# Running a Power Flow

We load the simple example network from the create_network tutorial from the pandapower.networks module:

In [27]:
import pandapower as pp
import pandapower.networks

net = pandapower.networks.example_simple()
net

This pandapower network includes the following parameter tables:
   - bus (7 elements)
   - shunt (1 elements)
   - ext_grid (1 elements)
   - switch (8 elements)
   - sgen (1 elements)
   - load (1 elements)
   - trafo (1 elements)
   - gen (1 elements)
   - line (4 elements)

## Run a Power Flow and Access Results

Runing a loadflow adds seperate result table with the prefix 'res_':

In [28]:
pp.runpp(net)

In [29]:
net

This pandapower network includes the following parameter tables:
   - bus (7 elements)
   - shunt (1 elements)
   - ext_grid (1 elements)
   - switch (8 elements)
   - sgen (1 elements)
   - load (1 elements)
   - trafo (1 elements)
   - gen (1 elements)
   - line (4 elements)
 and the following results tables:
   - res_ext_grid (1 elements)
   - res_gen (1 elements)
   - res_sgen (1 elements)
   - res_load (1 elements)
   - res_shunt (1 elements)
   - res_trafo (1 elements)
   - res_line (4 elements)
   - res_bus (7 elements)

These results tables are pandas datafarmes with the same index as the element table. For example, the bus table contains all bus voltages and summed bus power injections:

In [30]:
 net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,50.000000,6727.268201,7262.701273
1,1.020842,50.032006,0.000000,0.000000
2,1.020842,50.032006,0.000000,-1000.434322
3,1.024513,-98.316276,0.000000,0.000000
4,1.024513,-98.316276,0.000000,0.000000
5,1.030000,-98.251691,-6000.000000,-3513.228774
6,1.023156,-98.166885,-800.000000,2900.000000


We can now use pandas functionality to analyse the loadflow results, for example to get the minimum voltage in the medium voltage level:

In [31]:
net.res_bus[net.bus.vn_kv==20.].vm_pu.min()

1.0231555578398985

or the maxium voltage at a bus with load or generation:

In [32]:
load_or_generation_buses = set(net.load.bus.values) | set(net.sgen.bus.values) | set(net.gen.bus.values)
net.res_bus.vm_pu.loc[load_or_generation_buses].max()

1.0300000000000002

For more on how to use pandas for data analysis in pandapower, see the tutorial on [data analysis](data_analysis.ipynb).

## Result tables

Each element (except the switch) has its own result table with results tailored to the specific element. Here, we just show each table. For parameters definitions, see the documentation of the datastructure.

In [33]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,50.000000,6727.268201,7262.701273
1,1.020842,50.032006,0.000000,0.000000
2,1.020842,50.032006,0.000000,-1000.434322
3,1.024513,-98.316276,0.000000,0.000000
4,1.024513,-98.316276,0.000000,0.000000
5,1.030000,-98.251691,-6000.000000,-3513.228774
6,1.023156,-98.166885,-800.000000,2900.000000


In [34]:
net.res_ext_grid

,p_kw,q_kvar
0,6727.268201,7262.701273


In [35]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,-6727.268201,-7262.701328,6.730354e+03,1.570346e+03,3.086061,-5692.355559,0.050941,3.553340e-02,0.050941,8.663409
1,-5972.020868,-3572.601668,6.000000e+03,3.517662e+03,27.979108,-54.940094,0.196084,1.949296e-01,0.196084,46.575820
2,0.000024,-4.432793,9.641213e-08,6.042757e-08,0.000024,-4.432793,0.000124,3.188969e-12,0.000124,0.059160
3,800.000000,-2900.000000,-7.936175e+02,2.805748e+03,6.382452,-94.251942,0.084877,8.215876e-02,0.084877,20.160899


In [36]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.354263,-569.911445,6765.638417,766.853612,35.284154,196.942168,0.034728,0.191855,26.584187


In [37]:
net.res_load

,p_kw,q_kvar
0,1200.0,2400.0


In [38]:
net.res_sgen

,p_kw,q_kvar
0,-2000.0,500.0


In [39]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3513.228774,-98.251691,1.03


In [40]:
net.res_shunt

,p_kw,q_kvar,vm_pu
0,0.0,-1000.434322,1.020842


## Voltage Angles and Initialization

Maybe you wondered why even though there is a voltage angle of 50 degrees defined for the external grid: 

In [41]:
net.ext_grid.va_degree

0    50.0
Name: va_degree, dtype: float64

and a shift of 150° over the HV/MV transformer:

In [42]:
net.trafo.shift_degree

0    150.0
Name: shift_degree, dtype: float64

the voltage angles are all close to zero:

In [43]:
pp.runpp(net)
net.res_bus.va_degree

0    50.000000
1    50.032006
2    50.032006
3   -98.316276
4   -98.316276
5   -98.251691
6   -98.166885
Name: va_degree, dtype: float64

That is because the standard parameter for calculate_voltage_angles is False, which means voltage angles at external grids and transformer shifts are ignored by default. In a radial network, the absolute voltage angle shifts do not have an influence on the power flow, which is why they are disabled by default. In meshed networks however, where multiple external grids are galvanically coupled, it is always necessary to calculate the voltage angles.

Suppose we want to calculate the correct voltage angles and set calculate_voltage_angles to True:

In [44]:
pp.runpp(net, calculate_voltage_angles=True)

Now the power flow does not converge. This can happen with large angle shifts. The solution is to use a initialization with a DC loadflow instead of a flat start, which is default behaviour:

In [45]:
pp.runpp(net, calculate_voltage_angles=True, init="dc")

Now, we can see that all voltage angles are correctly calculated:

In [46]:
net.res_bus.va_degree

0    50.000000
1    50.032006
2    50.032006
3   -98.316276
4   -98.316276
5   -98.251691
6   -98.166885
Name: va_degree, dtype: float64

If we already have a solution, we can also initialize the loadflow with the voltage values from the last loadflow:

In [47]:
pp.runpp(net, calculate_voltage_angles=True, init="results")
net.res_bus.va_degree

0    50.000000
1    50.032006
2    50.032006
3   -98.316276
4   -98.316276
5   -98.251691
6   -98.166885
Name: va_degree, dtype: float64

The power flow converges and yields correct results where a flat start power flow would have failed.

Initializing with previous results can save convergence time in cases where multiple power flows with simliar input parameters are carried out consecutively, such as in quasi-static time series simulations.

## Transformer Model

The parameter "trafo_model" can be used to switch between a 'pi' and a 't' transformer model:

In [48]:
pp.runpp(net, trafo_model="t")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.354263,-569.911445,6765.638417,766.853612,35.284154,196.942168,0.034728,0.191855,26.584187


In [49]:
pp.runpp(net, trafo_model="pi")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.348659,-569.72374,6765.6392,766.663363,35.290541,196.939623,0.034728,0.191854,26.584104


For a definition of the different transformer model see the power flow model documentation of the transformer element.

## Transformer Loading

The transformer loading can either be calculated in relation to the rated current:

In [50]:
pp.runpp(net, trafo_loading="current")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.354263,-569.911445,6765.638417,766.853612,35.284154,196.942168,0.034728,0.191855,26.584187


or to the rated power of the transformer:

In [51]:
pp.runpp(net, trafo_loading="power")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6730.354263,-569.911445,6765.638417,766.853612,35.284154,196.942168,0.034728,0.191855,27.235837


The transformer loading does not have an influence on other power flow results besides the loading_percent parameter.

## Generator Reactive Power Limits

The generator has reactive power limits of -3000...3000 kvar:

In [52]:
net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type
0,generator,5,-6000.0,1.03,NaN,-3000.0,3000.0,1.0,True,None


which are however exceeded in the power flow results, because the enforce_q_lims option defaults to False:

In [53]:
pp.runpp(net)
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3513.228774,-98.251691,1.03


If the enforce_q_lims parameter is set to True, the reactive power limit is complied with, while the voltage deviates from the voltage set point of the generator:

In [54]:
pp.runpp(net, enforce_q_lims=True)
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3000.0,-98.225446,1.027434


If you want to know what to do when a power flow does not converge, continue with the [diagnostic tutorial](diagnostic.ipynb).

## Changing the Power Flow Algorithm

There are 5 algorithms available for solving the power flow problem:
* "nr" **Newton-Raphson** - default algorithm option
* "bfsw" **Backward/Forward Sweep** (specially suited for radial and weakly-meshed networks)
* "gs" **Gauss-Seidel** (pypower implementation)
* "fdbx" **Fast-Decoupled**  power flow using XB method (pypower implementation)
* "fdxb" **Fast-Decoupled**  power flow using BX method (pypower implementation)

Each algorithm can be selected by passing corresponding string {"nr", "bfsw", "gs", "fdbx", "fdxb"} to the parameter algorithm.  
For example, if you want to use the **Backward/Forward sweep** algorithm:

In [55]:
pp.runpp(net, algorithm="bfsw")

Or in the case of **Gauss-Seidel**:

In [56]:
pp.runpp(net, algorithm="gs")

If power flow is run without setting the algorithm parameter, **Newton-Raphson** will be used as the default algorithm option.

In [57]:
pp.runpp(net)

There is also possibility to select **maximum number of iterations** that will be used for the specific algorithm.  
In the following example max_iteration is set to 10, which is obviously not enough for the Gauss-Seidel to converge:

In [58]:
pp.runpp(net, algorithm="gs", max_iteration=10)

LoadflowNotConverged: Power Flow gs did not converge after 10 iterations!

## Setting User Options

It is possible to set user options that override the pandapower default parameters for one specific network. For the example network, the voltage angles are calculated by default:

In [59]:
pp.runpp(net)
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,50.000000,6727.268201,7262.701273
1,1.020842,50.032006,0.000000,0.000000
2,1.020842,50.032006,0.000000,-1000.434322
3,1.024513,-98.316276,0.000000,0.000000
4,1.024513,-98.316276,0.000000,0.000000
5,1.030000,-98.251691,-6000.000000,-3513.228774
6,1.023156,-98.166885,-800.000000,2900.000000


We now set the option calculate_voltage_angles to False with the set_user_pf_options function:

In [60]:
pp.set_user_pf_options(net, calculate_voltage_angles=False)

If we run another power flow without specifing parameters, the voltage angles are neglected:

In [61]:
pp.runpp(net)
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,0.000000,6727.268201,7262.701273
1,1.020842,0.032006,0.000000,0.000000
2,1.020842,0.032006,0.000000,-1000.434322
3,1.024513,1.683724,0.000000,0.000000
4,1.024513,1.683724,0.000000,0.000000
5,1.030000,1.748309,-6000.000000,-3513.228774
6,1.023156,1.833115,-800.000000,2900.000000


This change in standard behaviour is only valid for this one network.

When a parameter is specified directly in the runpp function, it overrides the user option:

In [62]:
pp.runpp(net, calculate_voltage_angles=True)
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,50.000000,6727.268201,7262.701273
1,1.020842,50.032006,0.000000,0.000000
2,1.020842,50.032006,0.000000,-1000.434322
3,1.024513,-98.316276,0.000000,0.000000
4,1.024513,-98.316276,0.000000,0.000000
5,1.030000,-98.251691,-6000.000000,-3513.228774
6,1.023156,-98.166885,-800.000000,2900.000000


The hierarchy for power flow options is therefore:
    1. Arguments passed to runpp
    2. User Options
    3. runpp default parameters